### ライブラリインポート

In [1]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger

import matplotlib.pyplot as plt
import os.path
import sys
import pprint
import numpy

#pprint.pprint(sys.path)

Using TensorFlow backend.


### 自前で用意したライブラリインポート ＋ データセットパス指定

In [2]:
from data import DataSet
from cnn_models import ResearchModels
import time

HOMEDIR = os.path.expanduser("~") + '/'
DATA_PATH = HOMEDIR + "dataset/UCF101/data_file.csv"
FRAME_DIR = HOMEDIR + "dataset/UCF101/UCF-101_frame"
TRAIN_DIR = FRAME_DIR + "/train"
TEST_DIR = FRAME_DIR + "/test"

data = DataSet(data_path=DATA_PATH)

### GPU制限(念の為)

In [3]:
import trainSetting 

trainSetting.GPU_Limit(0.9)
#trainSetting.GPU_LimitAllow()

### 学習を行う関数(参考コードそのまま)

In [4]:
def train(data_type, seq_length, model, saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
    # Helper: Save the model.
    checkpointer = ModelCheckpoint(
        filepath=os.path.join('data', 'checkpoints', model + '-' + data_type + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    # Helper: TensorBoard
    tb = TensorBoard(log_dir=os.path.join('data', 'logs', model))

    # Helper: Stop when we stop learning.
    early_stopper = EarlyStopping(patience=5)

    # Helper: Save results.
    timestamp = time.time()
    csv_logger = CSVLogger(os.path.join('data', 'logs', model + '-' + 'training-' + \
        str(timestamp) + '.log'))

    # Get the data and process it.
    if image_shape is None:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            data_path=DATA_PATH
        )
    else:
        data = DataSet(
            seq_length=seq_length,
            class_limit=class_limit,
            image_shape=image_shape,
            data_path=DATA_PATH
        )

    # Get samples per epoch.
    # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
    steps_per_epoch = (len(data.data) * 0.7) // batch_size

    if load_to_memory:
        # Get data.
        X, y = data.get_all_sequences_in_memory('train', data_type, FRAME_DIR)
        X_test, y_test = data.get_all_sequences_in_memory('test', data_type, FRAME_DIR)
    else:
        # Get generators.
        generator = data.frame_generator(batch_size, 'train', data_type, FRAME_DIR)
        val_generator = data.frame_generator(batch_size, 'test', data_type, FRAME_DIR)

    # Get the model.
    rm = ResearchModels(len(data.classes), model, seq_length, saved_model)

    # Fit!
    if load_to_memory:
        # Use standard fit.
        rm.model.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger],
            epochs=nb_epoch)
    else:
        # Use fit generator.
        rm.model.fit_generator(
            generator=generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger, checkpointer],
            validation_data=val_generator,
            validation_steps=40,
            workers=4)

### 各種パラメータ設定

In [5]:
def trainStart(model='conv_3d', saved_model=None,class_limit=None
               ,seq_length=40,load_to_memory=False,batch_size=24,nb_epoch=1):
    
    # Chose images or features and image shape based on network.
    if model in ['conv_3d', 'c3d', 'lrcn']:
        data_type = 'images'
        image_shape = (80, 80, 3)
    elif model in ['lstm', 'mlp']:
        data_type = 'features'
        image_shape = None
    else:
        raise ValueError("Invalid model. See train.py for options.")
    
    train(data_type, seq_length, model, saved_model=saved_model,
          class_limit=class_limit, image_shape=image_shape,
          load_to_memory=load_to_memory, batch_size=batch_size, nb_epoch=nb_epoch)

### 学習！！

In [6]:
trainStart(model='mlp',nb_epoch=100)

Loading simple MLP.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 81920)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               41943552  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
d

In [7]:
trainStart(model='lstm',nb_epoch=100)

Loading LSTM model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2048)              33562624  
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 101)               51813     
Total params: 34,663,525
Trainable params: 34,663,525
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
Creating test generator with 3418 samples.
Creating train generator with 8596 samples.
350/350 [==============================] - 70s 199ms/step - loss: 4.5301 - acc: 0.0333 - top_k_categorical_accuracy: 0.1204 - val_loss: 4.

350/350 [==============================] - 67s 192ms/step - loss: 0.8424 - acc: 0.7580 - top_k_categorical_accuracy: 0.9512 - val_loss: 1.0799 - val_acc: 0.7104 - val_top_k_categorical_accuracy: 0.9187

Epoch 00025: val_loss improved from 1.10899 to 1.07986, saving model to data/checkpoints/lstm-features.025-1.080.hdf5
Epoch 26/100
350/350 [==============================] - 67s 192ms/step - loss: 0.8103 - acc: 0.7695 - top_k_categorical_accuracy: 0.9512 - val_loss: 1.1631 - val_acc: 0.6875 - val_top_k_categorical_accuracy: 0.9146

Epoch 00026: val_loss did not improve from 1.07986
Epoch 27/100
350/350 [==============================] - 67s 192ms/step - loss: 0.7948 - acc: 0.7724 - top_k_categorical_accuracy: 0.9555 - val_loss: 1.0944 - val_acc: 0.7198 - val_top_k_categorical_accuracy: 0.9125

Epoch 00027: val_loss did not improve from 1.07986
Epoch 28/100
350/350 [==============================] - 67s 192ms/step - loss: 0.7621 - acc: 0.7812 - top_k_categorical_accuracy: 0.9588 - val_lo

In [6]:
trainStart(model='conv_3d',nb_epoch=100)

Loading Conv3D
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 38, 78, 78, 32)    2624      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 38, 39, 39, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 36, 37, 37, 64)    55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 36, 18, 18, 64)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 34, 16, 16, 128)   221312    
_________________________________________________________________
conv3d

In [6]:
trainStart(model='c3d',nb_epoch=100,batch_size=8)

Loading C3D
Instructions for updating:
Colocations handled automatically by placer.


/home/s-sato/notebook/library/cnn_models.py:204: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", input_shape=(40, 80, 8..., strides=(1, 1, 1), padding="same")`
  input_shape=self.input_shape))
/home/s-sato/notebook/library/cnn_models.py:206: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  border_mode='valid', name='pool1'))
/home/s-sato/notebook/library/cnn_models.py:210: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", strides=(1, 1, 1), padding="same")`
  subsample=(1, 1, 1)))
/home/s-sato/notebook/library/cnn_models.py:212: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
  border_mode='valid', name='pool2'))
/home/s-sato/notebook/library/cnn_model

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 40, 80, 80, 64)    5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 40, 40, 40, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 40, 40, 40, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 20, 20, 20, 128)   0         
_________________________________________________________________
conv3a (Conv3D)              (None, 20, 20, 20, 256)   884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 20, 20, 20, 256)   1769728   
___________________________